In [11]:
library(dplyr)
library(data.table)
library(tidyr)
library(dtplyr)
library(stringr)

In [12]:
cldf <- fread(snakemake@input$merged)

In [13]:
head(cldf)

In [14]:
df <- fread(snakemake@input$cluster)

In [15]:
data <- fread(snakemake@input$data, select = c("taxon","month"))

In [16]:
cldf2 <- cldf %>%
    left_join(data, by = c("Member"="taxon")) %>%
    mutate(Cluster = str_replace_all(Cluster,pattern = "'",replacement = "")) %>%
    left_join(df, by=c("Cluster"="SequenceName")) %>%
    as.data.frame()

In [17]:
head(cldf2)

In [18]:
mnths = unique(sort(data$month))
m1 = mnths[[1]]
m2 = mnths[[length(mnths)]]

In [23]:
df2 <- cldf2 %>%
    filter(month == m1 | month ==m2)
df3 <- df2  %>%
    group_by(ClusterNumber, month) %>%
    summarise(N=n()) %>%
    tidyr::pivot_wider(names_from = month, values_from = N) %>%
    na.omit() %>%
    rowwise() %>%
    mutate(total_nmb = sum(c_across(starts_with("20")))) %>%
    as.data.frame()
df3$ratio <- df3[,2] / df3[,3]
df3nonmin <- df3 %>%
    filter(ClusterNumber != -1)
df3$ratio <- df3[,2] / df3[,3]
good <- df3 %>%
    filter(total_nmb >= 200) %>%
    filter(ratio < 1, ratio > 0.1, ClusterNumber != -1)
# good <- df3 %>%
#     filter(total_nmb >= 200) %>%
#     filter(ratio < 1, ratio > 0.1, ClusterNumber != -1)
sd = sd(good$total_nmb)

out=data.frame(File = snakemake@input$cluster ,N=nrow(good), Nall=nrow(df3nonmin), snakemake@wildcards$p,
              meanv = mean(good$total_nmb, na.rm = TRUE), minv = min(good$total_nmb, na.rm = TRUE), maxv = max(good$total_nmb, na.rm = TRUE), sdv = sd(good$total_nmb, na.rm = TRUE))
fwrite(x = out, row.names = F,col.names = F, file = snakemake@output[[1]],na='NA')
out
